Analysis of our NRE v2 extractor's extractions on the finance data set (39,833 extractions from first 4000 documents).

In [1]:
import numpy as np
import pandas as pd
import unicodedata

In [2]:
# Read in data
columns = ["sentence", "arg1", "verb", "relation", "arg2", "value_1", "unit_1", "value_2", "unit_2", \
           "q_measure_1", "quantulum_v1", "quantulum_u1", "q_measure_2", "quantulum_v2", "quantulum_u2", "arg1_f", "rel_f"]

df = pd.read_csv('finance_NewExtractions.csv', names = columns)
len(df)

39833

In [4]:
# Only keep strings containing a capital letter (Assumption: interesting entities are proper nouns)
def capital_filter(text):
    if any(letter.isupper() for letter in str(text)):
        return text

In [7]:
# Apply filters
df['arg1_f'] = df.arg1_f.apply(lambda x: capital_filter(x))
df['value_1'] = pd.to_numeric(df['value_1'], errors='coerce')
df['value_2'] = pd.to_numeric(df['value_2'], errors='coerce')
df2 = df[df.value_1.notna() & df.arg1_f.notna()].reset_index(drop=True)
len(df2)

23055

In [8]:
# Standardize common units
df2.loc[df2['unit_1'] == 'percent', 'unit_1'] = '%'
df2.loc[df2['unit_1'] == 'per cent', 'unit_1'] = '%'
df2.loc[df2['unit_1'] == 'dollars', 'unit_1'] = '$'
df2.loc[(df2.rel_f == 'barrel') & (df2.unit_1 == '$'), 'unit_1'] = '$  per barrel'
df2.loc[df2['unit_1'] == '$  per barrel', 'unit_1'] = '$ per barrel'
df2.loc[(df2.rel_f == 'barrel') & (df2.unit_1 == '$ per barrel'), 'rel_f'] = np.NaN

In [9]:
# Group data on arg1, relation, and unit
# Count > 50
gdf1 = df2.groupby(["arg1_f", "rel_f","unit_1"])['value_1'].agg(["min","max","count"]).sort_values("count", ascending=False).query("count > 50")
gdf1[gdf1['min'] != gdf1['max']]

,,,min,max,count
arg1_f,rel_f,unit_1,,,
Dow Jones Industrial Average,points,%,0.01,2.670000e+00,293
Nasdaq Composite,points,%,0.00,3.500000e+00,252
S&P,points,%,0.01,1.440000e+00,100
Japan,MSCI broadest index,%,0.05,1.000000e+01,92
.DJI,points,%,0.01,3.290000e+00,89
Net income,share,$,10000000.00,1.904000e+10,64
Dow e - minis,points,%,0.02,1.740000e+00,51
Nasdaq 100 e - minis,points,%,0.01,2.400000e+00,51
S&P 500 e - minis,points,%,0.01,1.860000e+00,51


In [10]:
# Dollars per barrel: interquartile range of value
# Count > 10
gdf2 = df2.query("unit_1 == '$ per barrel'").groupby(["arg1_f","unit_1"])["value_1"].agg(["count"])
gdf2['25%'] = df2.groupby(["arg1_f","unit_1"])['value_1'].quantile(q=0.25)
gdf2['75%'] = df2.groupby(["arg1_f","unit_1"])['value_1'].quantile(q=0.75)
gdf2.query("count > 10").sort_values("count", ascending=False)

,,count,25%,75%
arg1_f,unit_1,,,
Brent crude futures,$ per barrel,84,63.8075,71.4425
U.S. crude,$ per barrel,82,53.7650,60.1700
U.S. West Texas Intermediate ( WTI ) crude futures,$ per barrel,60,54.1925,63.4075
Brent,$ per barrel,45,62.0100,71.4000
Brent crude,$ per barrel,41,66.9000,70.3800
U.S. crude futures,$ per barrel,28,55.7075,61.2225
Brent crude LCOc1 futures,$ per barrel,22,58.7875,63.7400
U.S. West Texas Intermediate ( WTI ) crude oil futures,$ per barrel,22,55.8625,57.3750
Brent futures,$ per barrel,17,62.2800,69.7200


In [18]:
# Nasdaq Composite stock value
# COUNT > 10
gdf3 = df2[df2.arg1.str.contains('Nasdaq Composite')].groupby(["verb","unit_1"])['value_1'].agg(["count"])
gdf3['25%'] = df2.groupby(["verb","unit_1"])['value_1'].quantile(q=0.25)
gdf3['75%'] = df2.groupby(["verb","unit_1"])['value_1'].quantile(q=0.75)
gdf3.query("count > 10")

count      25%     75%
verb     unit_1                           
added    %             96   0.2375   0.900
         points        86  19.0700  83.670
dropped  %             96   0.4600   3.200
         points        88  13.1100  67.790
gained   %             26   0.3000   3.400
         points        26   5.7900  27.770
recorded new highs     67  22.0000  58.250
         new lows      67   9.0000  37.250
was down %             50   0.1400   0.800
         points        50   8.4400  62.385
was up   %             66   0.2450   0.955
         points        67  13.0450  78.885

In [19]:
# Dow Jones Industrial Average stock value
# COUNT > 10
gdf3 = df2[df2.arg1.str.contains('Dow Jones Industrial Average')].groupby(["verb","unit_1"])['value_1'].agg(["count"])
gdf3['25%'] = df2.groupby(["verb","unit_1"])['value_1'].quantile(q=0.25)
gdf3['75%'] = df2.groupby(["verb","unit_1"])['value_1'].quantile(q=0.75)
gdf3.query("count > 10")

count      25%       75%
verb     unit_1                           
fell     %           92   0.6000    4.8000
         points      89  41.4525  190.1275
rose     %          101   0.5000    3.8250
         points      98  38.2575  174.4775
was      a.m. ET     37   9.0000   11.0000
         p.m. ET     25  12.0000   12.0000
was down %           57   0.1400    0.8000
         points      51   8.4400   62.3850
was up   %           60   0.2450    0.9550
         points      57  13.0450   78.8850

In [23]:
#Example sentence containing 'Dow Jones Industrial Average'.
df2.iloc[87].sentence

'The Dow Jones Industrial Average fell 17.78 points, or 0.06%, to 27,341.38, the S&P 500 fell 10.24 points, or 0.34%, to 3,004.06 and the Nasdaq Composite dropped 36.96 points, or 0.45%, to 8,221.23.'

#### Conclusions

Produces some interesting results, but difficult to get correct NRE for Financial news:

Relation noun-phrase is often missing - need to infer implicit relation which is not feasible in rule based system.

Unit may also be missing and require inference.

Entities (e.g. stock names/codes, fuels) are less likely to be correctly extracted.